<h3> Dense Optical Flow </h3>
<h4> Gunner Farnerback method </h4>

In [1]:
import cv2 as cv
import numpy as np
import collections
import sys

In [2]:
def getMovement(bgr):
    
    #get blue color value of all the pixels
    
    blueChannel = bgr[:,:,0].ravel()
    #greenChannel = bgr[:,:,1].ravel()
    redChannel = bgr[:,:,2].ravel()
    
    # create a dictionary of the frequency of color values
    # means 20 pixels have value 200 for blue
    # 10 pixels have value 100 for blue and so on and so forth
    blueCounts = collections.Counter(blueChannel)
    #greenCounts = collections.Counter(greenChannel)
    redCounts = collections.Counter(redChannel)

    
    # considering only those pixels which have values > 100 for any color
    blues = dict()
    reds = dict()
    #greens = dict()
    for key in range(100,256):
        blues[key] = blueCounts[key]
        #greens[key] = greenCounts[key]
        reds[key] = redCounts[key]
    
    
    # sum up the values of pixels for blue and red
    # if blue sum is more then there is more blue in the image
    # which implies more movement towards towards left and vice versa
    
    b=sum(blues.values())
    #g=sum(greens.values())
    r=sum(reds.values())
    
    # get maximum of blue and green
    m = max([b,r])
    if m == b:
        return "blue"
    #elif m == g:
        #return "green"
    else:
        return "red"

In [11]:
# this method needs review
def getMovement2(angle):
    angleDict = collections.Counter(angle.ravel())
    sum_left = 0
    sum_right = 0
    for k in angleDict:
        if k > 150 and k < 230:
            sum_left += angleDict[k]
        elif k < 40 and k > 320 and k!=0:
            sum_right += angleDict[k]
    if sum_right > sum_left:
        return 'red'
    else:
        return 'blue'

In [12]:
cv.namedWindow('Dense Optical Flow', cv.WINDOW_KEEPRATIO)

cap = cv.VideoCapture(0)

ret, frame1 = cap.read()

if ret:
    prevImg = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

    hsv_mask = np.zeros_like(frame1)

    # fully saturated image
    hsv_mask[:,:,1] = 255

    while True:

        ret, frame2 = cap.read()
        
        if ret:
            nextImg = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

            # 4th param pyramid scale (defined the scale of the next image in pyramid{keeping it half})
            # 5th param (levels of pyramid)
            # 6th param (winsize)
            # 7th param (iterations)
            # 8th param (poly_n size of pixel neighbourhood)
            # 9th param (poly_sigma standard deviation)
            # 10th param (flags)
            flow = cv.calcOpticalFlowFarneback(prevImg, nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)

            # flow returns the cartesian coordinates of the flow of pixels
            # we will convert them to polar coordinates (angle and magnitude)
            # where magnitude will then define "value" 
            # and angle will define "hue"

            mag, angle = cv.cartToPolar(flow[:, :, 0], flow[:, :, 1], angleInDegrees = True)

            # reducing the hue to half (so we'll have less colors)
            hsv_mask[:, :, 0] = angle/2

            hsv_mask[:, :, 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

            # convert the image back for visualization purpose
            bgr = cv.cvtColor(hsv_mask, cv.COLOR_HSV2BGR)

            move = getMovement(bgr)
            # this method needs review
            #move = getMovement2(angle)

            if move == "red":
                myFont = cv.FONT_HERSHEY_SIMPLEX
                cv.putText(img = bgr, text = 'Right', org = (5,175), fontFace = myFont, fontScale = 2,
                color = (100,100,100), thickness = 5, lineType = cv.LINE_AA)
            elif move == "blue":
                myFont = cv.FONT_HERSHEY_SIMPLEX
                cv.putText(img = bgr, text = 'Left', org = (5,175), fontFace = myFont, fontScale = 2,
                color = (100,100,100), thickness = 5, lineType = cv.LINE_AA)
            x = cv.addWeighted(frame2, 0.3, bgr, 0.7, 10)

            cv.imshow('Dense Optical Flow', x)

            if cv.waitKey(10) & 0xFF == 27:
                break

            prevImg = nextImg
else:
    print('cannot read form video')
cap.release()

cv.destroyAllWindows()

In [6]:
cap.release()

cv.destroyAllWindows()